<a href="https://colab.research.google.com/github/vmcarranzam/Proyecto_RL_CDP/blob/main/Regresi%C3%B3n_Lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto Regresión Lineal con Python

##Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colormaps as cm
import seaborn as sns
from sklearn.linear_model import LinearRegression
from google.colab import drive
drive.mount('/content/drive')

##Carga de datos

In [ ]:
data=np.load("/content/drive/MyDrive/Colab Notebooks/Proyecto Ciencia de Datos/proyecto_training_data.npy")
df=pd.DataFrame(data, columns=["SalePrice","OverallQual","FirstFlrSF", "TotRmsAbvGrd","YearBuilt","LotFrontage"])
df

##Definición de la clase

In [ ]:
class RegresionLineal:
    
    def __init__(self, x_col, y_col, epochs, learning_rate, imprimir_cada, df):
        self.x_col = x_col
        self.y_col = y_col
        self.x = np.array(df[x_col])
        self.y = np.array(df[y_col])
        self.epochs = epochs
        self.lr = learning_rate
        self.imprimir_cada = imprimir_cada
        self.train_x, self.test_x, self.train_y, self.test_y = self.split_data()
        self.mse = {}

 #Método para dividir el dataset   
    def split_data(self):
        idx = int(len(self.x) * 0.8)
        train_x = self.x[:idx]
        test_x = self.x[idx:]
        train_y = self.y[:idx]
        test_y = self.y[idx:]
        return train_x, test_x, train_y, test_y

#Método para graficar histogramas de las variables independientes
    def plot_histograms(self):
        sns.set_style("darkgrid")
        for column in df:
          plt.clf()
          col = (np.random.random(), np.random.random(), np.random.random())
          plt.figure()
          sns.histplot(df[column], kde = True, color = col, linewidth = 1, edgecolor ="0")
          plt.show()

#Método para realizar regresión lineal y gradiente descendiente    
    def perform_vectorized_linear_regression(self):
        b0, b1 = 0, 0
        n = len(self.train_x)
        for i in range(self.epochs):
            y_pred = b0 + b1 * self.train_x
            error = y_pred - self.train_y
            b0 -= self.lr * (1/n) * error.sum()
            b1 -= self.lr * (1/n) * np.dot(error, self.train_x)
            if (i+1) % self.print_every == 0:
                self.mse[i+1] = {'b0': b0, 'b1': b1, 'y_pred': y_pred, 'mse': np.mean(error**2)}
                print("Epoch: {}, MSE: {}".format(i+1, self.mse[i+1]["mse"]))